In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import pyfolio as pf
import quantstats as qs
import mplfinance as mpf
import pyarrow as pa
import pyarrow.parquet as pq
import talib as ta
import importlib

/Users/Jason/work/source/03_ThorpAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import sys
import os
from pathlib import Path

def find_project_root(project_name):
    current = Path(os.getcwd()).resolve()
    while current != current.parent:  # 向上查找直到根目录
        if project_name in str(current.name):  # 使用精确匹配
            return str(current)
        current = current.parent
    raise Exception(f"找不到名为 '{project_name}' 的目录")

project_root = find_project_root('ThorpAI')

# 将 project_root 根目录添加到系统路径
sys.path.append(project_root)

In [3]:
from btc_model.core.update_data.okx_downloader import OKxDownloader

from btc_model.setting.setting import get_settings
from btc_model.core.common.const import PROJECT_NAME, Interval
from btc_model.core.util.file_util import FileUtil


from btc_model.core.common.const import PROJECT_NAME
from btc_model.core.common.const import (Exchange,
                                         Interval,
                                         InstrumentType,
                                         Product,
                                         ProviderType,
                                         EntityType
                                         )
from btc_model.core.common.context import Context
from btc_model.core.update_data.okx_downloader import OKxDownloader
from btc_model.core.update_data.binance_downloader import BinanceDownloader
from btc_model.core.data_loader.crypto_data_loader import CryptoDataLoader
from btc_model.core.util.log_util import Logger
from btc_model.core.util.file_util import FileUtil
from btc_model.setting.setting import get_settings

from btc_model.indicator.base_indicator import BaseIndicator
from btc_model.indicator.indicator_bollinger import IndicatorBollinger
from btc_model.indicator.indicator_macd import IndicatorMACD
from btc_model.indicator.indicator_mayer_multiple import IndicatorMayerMultiple
from btc_model.indicator.indicator_pi_cycle import IndicatorPiCycle
from btc_model.indicator.indicator_rsi import IndicatorRSI


In [4]:
data_directory = FileUtil.get_project_dir(project_name=PROJECT_NAME, sub_dir='data')

In [5]:
data_loader = CryptoDataLoader(data_directory)
df_instruments = data_loader.load_instruments(exchange=Exchange.OKX)
df_indicator = data_loader.load_indicator_data(exchange=Exchange.OKX, interval=Interval.DAILY, provider_type=ProviderType.OKX)
df_indicator = df_indicator[df_indicator['symbol_id'] == 'BTC-USDT']

In [ ]:
df_indicator

In [14]:
import pandas as pd
import plotly.graph_objects as go

df = df_indicator.copy()
if df.empty:
    raise ValueError("No data available in df_indicator")

# Calculate moving averages with error handling
df['111DMA'] = df['close'].rolling(window=111, min_periods=1).mean()
df['350DMA'] = df['close'].rolling(window=350, min_periods=1).mean()
df['350DMA_x2'] = df['350DMA']*2

# Create figure with improved styling
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['datetime'],
    y=df['close'],
    name='BTC价格',
    line=dict(color='gray', width=1)
))
fig.add_trace(go.Scatter(
    x=df['datetime'],
    y=df['111DMA'],
    name='111DMA',
    line=dict(color='green', width=1.5)
))
fig.add_trace(go.Scatter(
    x=df['datetime'],
    y=df['350DMA']*2,
    name='350DMA×2',
    line=dict(color='purple', width=1.5)
))

# Enhanced layout
fig.update_layout(
    title='Pi Cycle Top 指标',
    xaxis_title='日期',
    yaxis_title='价格（美元）',
    xaxis=dict(
        hoverformat='%Y-%m-%d'  # 设置悬停时显示的日期格式
    ),
    hovermode='x unified',
    template='plotly_white',
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)


# Show with error handling
try:
    fig.show()
except ValueError as e:
    print(f"Error displaying figure: {e}")
    print("Please ensure nbformat>=4.2.0 is installed")

In [15]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# 数据准备
source = ColumnDataSource(df)
p = figure(x_axis_type="datetime", title="Pi Cycle Top 指标", width=800, height=400)
p.line('datetime', 'close', source=source, legend_label='BTC价格', line_color='gray')
p.line('datetime', '111DMA', source=source, legend_label='111DMA（绿线）', line_color='green')
p.line('datetime', '350DMA_x2', source=source, legend_label='350DMA×2（紫线）', line_color='purple')

p.legend.location = "top_left"
show(p)


In [14]:
from btc_model.setting.setting import get_settings
from btc_model.core.wrapper.okx_api_wrapper import OKxApiWrapper

setting = get_settings('cex.okx')

apikey = setting['apikey']
secretkey = setting['secretkey']
passphrase = setting['passphrase']
proxy = setting['proxy']

instance = OKxApiWrapper.get_instance(apikey, secretkey, passphrase, proxy)

start_dt = '2011-01-01 00:00:00'
end_dt = '2023-02-26 23:59:59'
result = instance.get_history_index_kline_data("BTC-USD", Interval.DAILY_UTC, start_dt, end_dt)

# result = instance.get_index_ticker(symbol_id='BTC-USD')


print(pd.DataFrame(result))


INFO   : 2025-02-26 21:55:50,074 >>> HTTP Request: GET https://www.okx.com/api/v5/market/index-candles?instId=BTC-USD&after=1677455999000&before=1293840000000&bar=1Dutc&limit=100 "HTTP/2 200 OK"
INFO   : 2025-02-26 21:55:50,221 >>> HTTP Request: GET https://www.okx.com/api/v5/market/index-candles?instId=BTC-USD&after=1668816000000&before=1293840000000&bar=1Dutc&limit=100 "HTTP/2 200 OK"
INFO   : 2025-02-26 21:55:50,364 >>> HTTP Request: GET https://www.okx.com/api/v5/market/index-candles?instId=BTC-USD&after=1660176000000&before=1293840000000&bar=1Dutc&limit=100 "HTTP/2 200 OK"
INFO   : 2025-02-26 21:55:50,509 >>> HTTP Request: GET https://www.okx.com/api/v5/market/index-candles?instId=BTC-USD&after=1651536000000&before=1293840000000&bar=1Dutc&limit=100 "HTTP/2 200 OK"
INFO   : 2025-02-26 21:55:50,661 >>> HTTP Request: GET https://www.okx.com/api/v5/market/index-candles?instId=BTC-USD&after=1642896000000&before=1293840000000&bar=1Dutc&limit=100 "HTTP/2 200 OK"
INFO   : 2025-02-26 21:55

    symbol_id   datetime      open      high       low     close
0     BTC-USD 2021-03-20  58087.78  59890.73  57853.14  58119.87
1     BTC-USD 2021-03-21  58121.79  58650.68  55533.12  57385.48
2     BTC-USD 2021-03-22  57388.27  58430.61  53752.50  54103.78
3     BTC-USD 2021-03-23  54103.75  55852.96  52988.77  54370.24
4     BTC-USD 2021-03-24  54366.52  57212.19  51693.39  52286.36
..        ...        ...       ...       ...       ...       ...
704   BTC-USD 2023-02-22  24452.90  24471.50  23591.00  24188.40
705   BTC-USD 2023-02-23  24188.40  24597.30  23625.40  23941.80
706   BTC-USD 2023-02-24  23941.80  24130.80  22837.30  23193.60
707   BTC-USD 2023-02-25  23193.60  23216.20  22773.10  23167.60
708   BTC-USD 2023-02-26  23167.60  23680.60  23073.20  23560.00

[709 rows x 6 columns]


In [20]:
from btc_model.setting.setting import get_settings
from btc_model.core.wrapper.binance_api_wrapper import BinanceApiWrapper

setting = get_settings('cex.binance')

apikey = setting['apikey']
secretkey = setting['secretkey']

instance = BinanceApiWrapper.get_instance(apikey, secretkey)

# instruments = instance.get_instruments(product=Product.SPOT)
# print(instruments)

start_dt = '2011-01-01 00:00:00'
end_dt = '2025-01-18 23:59:59'
result = instance.get_history_index_kline_data("USDT-BTC", Interval.DAILY, start_dt, end_dt)

print(result)


BinanceAPIException: APIError(code=-1102): Mandatory parameter 'pair' was not sent, was empty/null, or malformed.